Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
#path to pickle of all our datasets from the previous assignment. if not present run code in assignment1.ipynb
pickle_file = '../From Machine Learning to Deep Learning/notMNIST/notMNIST.pickle'

#open and load pickle file. extract each one of the datasets 
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

(old model: y = WX + b)
- data as a flat matrix ,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]. 
    # i.e. np.arange creates array from 0 to 9, compares each index in that list to each index in the 1D labels array
    # 0 as [True, False, False,...], 1 as [False, True, False,...] then cast all as float
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

#reshape and reformat datasets and labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below. i.e. create the neural network to be used, what it will receive, how it will store them, and what operations will be performed:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below i.e. runs your i.e. runs your input through the neural networks and outputs your predictions :

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    # creates constant tensor to hold values of datasets
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    # draw random values from truncated normal distribution with mean 0.0, stddev 1.0.
    #outputs float32 tensor of shape (784, 10)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    # 1D array of zeroes. shape(10, )
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    # run operation WX + b where X is input from your dataset
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [6]:
num_steps = 801

#calculate accuracy from training
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.492252
Training accuracy: 9.3%
Validation accuracy: 12.4%
Loss at step 100: 2.322866
Training accuracy: 71.4%
Validation accuracy: 71.3%
Loss at step 200: 1.891486
Training accuracy: 74.2%
Validation accuracy: 73.5%
Loss at step 300: 1.644658
Training accuracy: 75.7%
Validation accuracy: 74.3%
Loss at step 400: 1.474908
Training accuracy: 76.3%
Validation accuracy: 74.6%
Loss at step 500: 1.348414
Training accuracy: 76.9%
Validation accuracy: 74.8%
Loss at step 600: 1.249825
Training accuracy: 77.4%
Validation accuracy: 75.1%
Loss at step 700: 1.170308
Training accuracy: 77.9%
Validation accuracy: 75.5%
Loss at step 800: 1.104381
Training accuracy: 78.6%
Validation accuracy: 75.7%
Test accuracy: 82.4%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.726379
Minibatch accuracy: 9.4%
Validation accuracy: 13.9%
Minibatch loss at step 500: 2.589379
Minibatch accuracy: 66.4%
Validation accuracy: 75.6%
Minibatch loss at step 1000: 1.354537
Minibatch accuracy: 76.6%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 1.386003
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.959894
Minibatch accuracy: 82.0%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 1.107547
Minibatch accuracy: 78.9%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 0.798675
Minibatch accuracy: 74.2%
Validation accuracy: 79.5%
Test accuracy: 86.8%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [9]:
# From notes from course:
# a simple 2 layer neural network consists of a set of weights and biases applied to X (WX + b) 
# whose output is passed through H number of ReLUs. The output of passing through RELUs is passed 
# to the next set of weights and biases applied to this intermediate output. The output of the RELUs 
# is not visible outside of the network so these layers of RELUs are called hidden layers. 
# Whole process so far (WX + b) -> z -> RELUs(z) -> I -> WI + b -> y -> Softmax(y) -> scores (0-1)
# where z is the result of a matrix multiply, I is a RELUs intermediatary output, and y are logits

#set up our neural network with 1024 hidden nodes

batch_size = 128
h_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Constants
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, h_nodes]))
    b1 = tf.Variable(tf.zeros([h_nodes]))
    W2 = tf.Variable(tf.truncated_normal([h_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))    

    # Training computation.
    L1 = tf.matmul(tf_train_dataset, W1) + b1
    h_layer = tf.nn.relu(L1)
    logits = tf.matmul(h_layer, W2) + b2 
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    L1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(L1_valid, W2) + b2)
    L1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(L1_test, W2) + b2)

In [ ]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 359.436157
Minibatch accuracy: 13.3%
Validation accuracy: 32.5%
Minibatch loss at step 500: 21.333145
Minibatch accuracy: 72.7%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 10.648014
Minibatch accuracy: 79.7%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 8.131924
Minibatch accuracy: 76.6%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 5.103513
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 6.314469
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 2.609656
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 3500: 2.083629
Minibatch accuracy: 86.7%
Validation accuracy: 82.4%
Minibatch loss at step 4000: 1.178059
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 4500: 1.060502
Minibatch accuracy: 89.1%
Validation accuracy: 83.1%
Minibatch loss at step 5000: 1.753566
Minibatch accuracy: 85.2%
Validation acc